# New Store Business in Toronto

## Introduction/Business Problem

Our client is a commercial group (family business) and its current business focuses on various stores across North America, such as Clothing Stores, Health Food Stores, Beer 
Stores, etc. The client/company wants to expand its business into Toronto city and our goal is to define the right business 
expansion strategy for the company.

* What kind of business/store should the company consider in order to achieve the most possible success? 
* Which district in Toronto city should the Store be located? 

![Image of Store Type](http://2bsiwtsz6ylibiex20hddm1c-wpengine.netdna-ssl.com/wp-content/uploads/2013/04/mapoftorontoneighbourhoods.jpg)
<center>the Best Neighborhoods to do Business in Toronto By iQ Office Suites</center>

## Data acquisition and processing

We have already generated the venue matrix of Toronto city using Foursquaresoup API and data scraped from the wikipedia webpage:

https://developer.foursquare.com/places-api

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

We can process the venue matrix and group rows by neighborhood and by taking the sum of the frequency of occurrence of each category.
By adding the frequency of ocurrence of each store type in different districts, we can get the total number for each store type in Toronto city.
Once we have the distribution of store type, we can suggest which type of store is the most common and what kind of store is mostly welcomed recently.

From the Statistics of Canada, we can also easily get the data about population census in each district of Toronto.
By analyzing the population relationship with different combination of store type, we can define the store type in the exact district to cater for its potential future population growth.

## Methodology 